In [1]:
import pandas as pd
import os
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action="ignore")

# for made indexinf in PCA
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition

%matplotlib inline

In [2]:
year = 2014
path = f"C:/Users/Redata/Downloads/read/kemiskinan/hh{year}/"

## Primary Independent variable

### Expenditure approach

In [3]:
# time in t-1
consum07 = pd.read_stata(path + "pce07nom.dta", index_col="hhid07")
kon = pd.DataFrame()
for i in consum07.columns:
    if i not in ["commid07","owners","hhid07_9"]:
        consum07[i] = consum07[i].astype("float64")
        kon[i] = consum07[i]

kon["kabid"] = kon["kabid"].fillna(1)
kon.fillna(0, inplace=True)

#HH expenditure per capita
kon["nonfood_t1"] = consum07[["xnonfood2", "xnonfood3","xhouse", "xeducall", 
                           "inonfood"]].fillna(0).sum(axis=1) / kon["hhsize"] # we include transfer and own produce
kon["food_t1"] = consum07[["xstaple","xvege","xdried",
                         "xmeat","xfish","xdairy","xspices","xsugar","xoil","xbeve",
                         "xaltb","xsnack","xfdout"]].sum(axis=1) / kon["hhsize"]
kon["pce_t1"] = kon["food_t1"] + kon["nonfood_t1"]
kon.reset_index(inplace=True)

In [4]:
df = pd.read_stata(f"dataneetpertama{year}.dta")

data = pd.merge(df, kon[["nonfood_t1", "food_t1", "pce_t1", "hhid07", "kabid"]], 
                left_on="hhid07", right_on="hhid07", how="left")
data.rename(columns={"kabid":"kabid_t1"}, inplace=True)

#if Nan covert as indonesia location
data["kabid_t1"] = data["kabid_t1"].fillna(1)
df.shape

(4909, 27)

In [5]:
# time in t
consum14 = pd.read_stata(path + "pce14nom.dta", index_col="hhid14")
kon = pd.DataFrame()
for i in consum14.columns:
    if i not in ["commid14","owners","hhid14_9"]:
        consum14[i] = consum14[i].astype("float64")
        kon[i] = consum14[i]
        
kon["kabid"] = kon["kabid"].fillna(1)
kon.fillna(0, inplace=True)

#HH expenditure per capita
kon["nonfood_t"] = consum14[["xnonfood2", "xnonfood3","xhouse", "xeducall", 
                           "inonfood"]].fillna(0).sum(axis=1) / kon["hhsize"] # we include transfer and own produce
kon["food_t"] = consum14[["xstaple","xvege","xdried",
                         "xmeat","xfish","xdairy","xspices","xsugar","xoil","xbeve",
                         "xaltb","xsnack","xfdout"]].sum(axis=1) / kon["hhsize"]
kon["pce_t"] = kon["food_t"] + kon["nonfood_t"]
kon.reset_index(inplace=True)


data = pd.merge(data, kon[["nonfood_t", "food_t", "pce_t", "hhid14", "kabid"]], 
                left_on="hhid14", right_on="hhid14", how="left")

#if Nan covert as indonesia location
data["kabid"] = data["kabid"].fillna(1)
data.shape

(4909, 35)

In [6]:
data.sample(10)

,index,sex,rspndnt,age,marstat,pidlink,ar02b_14,hhid14,pid14,hhid07,...,NEET__just_other_,year,nonfood_t1,food_t1,pce_t1,kabid_t1,nonfood_t,food_t,pce_t,kabid
2868,2868,1:Male,3,17.0,1:Not yet married,188180007,3,1881800,7.0,1881800,...,0,2014,505683.334375,175586.666406,681270.000781,3372.0,313273.331250,191750.000781,505023.332031,3372.0
3331,3331,3:Female,3,23.0,2:Married,228260005,3,2282600,5.0,2282600,...,0,2014,84493.054688,109922.222656,194415.277344,3508.0,298050.000391,455433.328906,753483.329297,3508.0
2648,2648,1:Male,3,16.0,1:Not yet married,177090006,3,1770900,6.0,1770900,...,0,2014,112635.417969,329549.996582,442185.414551,3320.0,368666.668750,296399.997070,665066.665820,3320.0
1040,1040,3:Female,3,24.0,1:Not yet married,062070005,3,0620700,5.0,0620700,...,0,2014,261250.005208,169144.442708,430394.447917,1808.0,184277.778646,387111.104818,571388.883464,1812.0
2163,2163,1:Male,3,22.0,1:Not yet married,142213103,3,1422131,3.0,1422131,...,0,2014,83033.333594,162933.330664,245966.664258,3603.0,196966.665625,507866.668750,704833.334375,3603.0
997,997,1:Male,3,18.0,1:Not yet married,060120005,3,0601200,5.0,0601200,...,0,2014,126652.779948,152244.445312,278897.225260,1805.0,352555.557292,295100.007080,647655.564372,1805.0
705,705,3:Female,3,15.0,1:Not yet married,042060014,3,0420600,14.0,0420600,...,0,2014,39607.142299,124738.094866,164345.237165,1609.0,114055.554688,176222.217448,290277.772135,1609.0
1815,1815,1:Male,3,20.0,1:Not yet married,122060010,3,1220600,10.0,1220600,...,0,2014,104513.890625,75255.556315,179769.446940,3209.0,314107.145089,280304.760603,594411.905692,3172.0
2816,2816,1:Male,3,16.0,1:Not yet married,185195103,3,1851951,3.0,1851951,...,0,2014,NaN,NaN,NaN,1.0,393583.335938,261083.331055,654666.666992,3329.0
1021,1021,3:Female,3,16.0,1:Not yet married,061100008,3,0611000,8.0,0611000,...,0,2014,73888.889323,174272.221029,248161.110352,1807.0,272708.333333,343055.558594,615763.891927,1807.0


### Household Asset approach

In [7]:
hr = pd.read_stata(path + "b3a_hr1.dta")[["hr1type",  "hhid14"]] #"hr01", "hr02", "hr05",
hr["ss"] = 0
hr0 = pd.pivot_table(hr, index=['hhid14'],
                    columns=['hr1type'], aggfunc=np.min).replace(0, np.nan)
hr0.columns = [i for i in hr.hr1type.unique()]

###only for 2014 dataset
hr = pd.read_stata(path + "b3a_hr1.dta")[["hr02_" + i.lower() for i in hr.hr1type.unique()] + ["hhid14"]].groupby("hhid14").min()
hr.columns = [i.split("_")[-1].upper() for i in hr.columns]

for i in hr0.columns:
    hr0[i] = hr[i]
###    

hr0 = hr0[hr0.columns[hr0.isnull().sum()/len(hr0) < 0.35]]
hr0.columns = [i+"_hr1type_t" for i in hr0.columns]

hr = pd.read_stata(path + "b3a_hr1.dta")[["hr1type", "hr01",  "hhid14"]] #"hr01", "hr02", "hr05",
hr1 = pd.pivot_table(hr, index=['hhid14'],
                    columns=['hr1type'], aggfunc=np.min).replace(0, np.nan)
hr1.columns = [i+"_hr1type_t" for i in hr.hr1type.unique()]

In [8]:
for i in hr0.columns:
    hr1[i] = (hr1[i] + hr0[i].fillna(0)).apply(lambda x : np.nan if x == 1  else x)

In [9]:
data = pd.merge(data, hr1[["A_hr1type_t", "E_hr1type_t", "F_hr1type_t"]], 
                left_on="hhid14", right_index=True, how="left")
data.sample(3)

,index,sex,rspndnt,age,marstat,pidlink,ar02b_14,hhid14,pid14,hhid07,...,food_t1,pce_t1,kabid_t1,nonfood_t,food_t,pce_t,kabid,A_hr1type_t,E_hr1type_t,F_hr1type_t
3141,3141,3:Female,3,18.0,1:Not yet married,209160009,3,2091651,4.0,2091600,...,526066.674805,1.195089e+06,3403.0,1.055458e+06,576333.334961,1.631792e+06,3276.0,200000001.0,15000001.0,1900001.0
160,160,1:Male,3,19.0,1:Not yet married,009210009,3,0092100,9.0,0092100,...,135777.777262,3.220681e+05,1210.0,2.598125e+05,185249.996582,4.450625e+05,1210.0,NaN,NaN,NaN
1284,1284,1:Male,3,17.0,1:Not yet married,082023103,3,0820231,3.0,0820231,...,252958.334635,4.004167e+05,3671.0,9.905278e+05,428277.780273,1.418806e+06,3671.0,NaN,NaN,NaN


In [10]:
path_t = f"C:/Users/Redata/Downloads/read/kemiskinan/hh{year-7}/"

hr = pd.read_stata(path_t + "b3a_hr1.dta")[["hr1type", "hr02",  "hhid07"]] #"hr01", "hr02", "hr05",
hr0 = pd.pivot_table(hr, index=['hhid07'],
                    columns=['hr1type'], aggfunc=np.min).replace(0, np.nan)
hr0.columns = [i+"_hr1type_t1" for i in hr.hr1type.unique()]
hr0 = hr0[hr0.columns[hr0.isnull().sum()/len(hr1) < 0.35]]

hr = pd.read_stata(path_t + "b3a_hr1.dta")[["hr1type", "hr01",  "hhid07"]] #"hr01", "hr02", "hr05",
hr1 = pd.pivot_table(hr, index=['hhid07'],
                    columns=['hr1type'], aggfunc=np.min).replace(0, np.nan)
hr1.columns = [i+"_hr1type_t1" for i in hr.hr1type.unique()]

for i in hr0.columns:
    hr1[i] = (hr1[i] + hr0[i].fillna(0)).apply(lambda x : np.nan if x == 1  else x)

In [11]:
data = pd.merge(data, hr1[["A_hr1type_t1", "E_hr1type_t1", "F_hr1type_t1"]], 
                left_on="hhid07", right_index=True, how="left")
data.sample(3)

,index,sex,rspndnt,age,marstat,pidlink,ar02b_14,hhid14,pid14,hhid07,...,nonfood_t,food_t,pce_t,kabid,A_hr1type_t,E_hr1type_t,F_hr1type_t,A_hr1type_t1,E_hr1type_t1,F_hr1type_t1
2754,2754,3:Female,3,15.0,1:Not yet married,182080007,3,1821841,3.0,1821841,...,3.721402e+06,461500.000977,4.182902e+06,3375.0,300000001.0,20000001.0,5000001.0,3.0,2700001.0,1640001.0
4783,4783,1:Male,3,16.0,1:Not yet married,314200006,3,3142000,6.0,3142000,...,3.391250e+05,498874.997070,8.380000e+05,7313.0,50000001.0,22500001.0,3500001.0,3.0,1000001.0,3.0
2233,2233,1:Male,3,23.0,3:Separated,145180005,3,1451800,5.0,1451800,...,2.093667e+05,323613.329297,5.329800e+05,3604.0,100000001.0,4000001.0,2000001.0,50000001.0,13000001.0,2295001.0


In [12]:
data.shape

(4909, 41)

## Value Adjustment to CPI

In [13]:
# for t-1
ihk = pd.read_csv("C:/Users/Redata/Downloads/cpi indonesia after modelling 2000 - 2016 (511 cities inflation).csv")
ihk = ihk[(ihk["year"] == 2007) & (ihk["jenis"] == "Umum")][["nmkab2014", "IHK", "kabid07", "latitude", "longitude"]]
ihk = ihk.groupby("kabid07", as_index=False).min()

data = pd.merge(data, ihk[["kabid07", "IHK"]].rename(columns={"IHK":"IHK_t1"}), 
                left_on="kabid_t1", right_on="kabid07", how="left")

# for in t
ihk = pd.read_csv("C:/Users/Redata/Downloads/cpi indonesia after modelling 2000 - 2016 (511 cities inflation).csv")
ihk = ihk[(ihk["year"] == 2014) & (ihk["jenis"] == "Umum")][["nmkab2014", "IHK", "kabid14"]]
ihk = ihk.groupby("kabid14", as_index=False).min()

data = pd.merge(data, ihk[["kabid14", "IHK", "nmkab2014"]], left_on="kabid", right_on="kabid14", how="left")

In [14]:
data.shape

(4909, 46)

In [15]:
#filled missing CPI with Indonesia's CPI
data.IHK = data.IHK.fillna(118.06)
data.IHK_t1 = data.IHK_t1.fillna(76.21)

In [16]:
data.drop(["kabid07", "kabid14"], axis=1, inplace=True)

In [17]:
data.sample(5)

,index,sex,rspndnt,age,marstat,pidlink,ar02b_14,hhid14,pid14,hhid07,...,kabid,A_hr1type_t,E_hr1type_t,F_hr1type_t,A_hr1type_t1,E_hr1type_t1,F_hr1type_t1,IHK_t1,IHK,nmkab2014
288,288,1:Male,3,16.0,1:Not yet married,018120006,3,0181200,6.0,0181200,...,1272.0,NaN,NaN,NaN,NaN,3.0,3000001.0,76.82,122.03,TANJUNG BALAI
1397,1397,3:Female,3,18.0,1:Not yet married,091060009,3,0910631,3.0,0910631,...,3671.0,NaN,NaN,NaN,40000001.0,8000001.0,2000001.0,78.02,122.99,TANGERANG
2644,2644,3:Female,3,22.0,1:Not yet married,176190013,3,1761900,13.0,1761900,...,3320.0,NaN,NaN,NaN,70000001.0,4800001.0,5000001.0,79.40,123.84,JEPARA
2229,2229,1:Male,3,15.0,1:Not yet married,145150008,3,1451500,8.0,1451500,...,3604.0,NaN,NaN,NaN,50000001.0,3.0,1300001.0,76.35,120.63,SERANG
3586,3586,3:Female,3,15.0,1:Not yet married,245283103,3,2452831,3.0,2452831,...,3523.0,3.0,150001.0,NaN,NaN,3.0,3.0,76.31,116.01,TUBAN


### Storing data

In [18]:
data.to_stata(f"dataneetkedua{year}.dta", write_index=False)